In [7]:
!pip install tensorflow transformers pandas tqdm scikit-learn tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 210.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 318.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 409.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 238.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 493.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 357.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 139.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 546.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 431.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 215.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 522.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 135.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

('snli_1.0_train.ko.tsv', <http.client.HTTPMessage at 0x70c5d7688050>)

In [12]:
# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

('xnli.dev.ko.tsv', <http.client.HTTPMessage at 0x70c82789ddd0>)

In [13]:
# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x70c82789ead0>)

In [14]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [15]:
train_snli.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


In [16]:
train_xnli.head()

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [17]:
# 결합 후 섞기
train_data = pd.concat([train_snli, train_xnli], ignore_index=True)
train_data = train_data.sample(frac=1)

In [18]:
train_data.head()

,sentence1,sentence2,gold_label
658601,"그도 최고야, 커비 켈스는 잘 씹은 담배를 한쪽 뺨에서 다른 쪽 뺨으로 옮겼다.",켈스는 특정 브랜드의 담배만 씹었다.,neutral
92087,곡선 구조 위에 있는 익스트림 스포츠 스키 선수,스키 선수가 내리막길을 걷고 있다.,entailment
168414,"반다나를 든 여자, 빨간 모자를 쓴 남자, 안경과 지팡이를 든 남자가 벤치에 앉아 ...",빨간 모자를 쓴 남자와 안경과 지팡이를 든 남자는 같은 사람이다.,contradiction
98616,행사는 일본 사원에서 열린다.,그 행사는 야구장에서 열리고 있다.,contradiction
919314,시간은 15분이다.,시간은 15분이었다.,entailment


In [19]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [20]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [21]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [22]:
# 결측값 및 중복 샘플 제거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)

In [23]:
train_data

,sentence1,sentence2,gold_label
0,"그도 최고야, 커비 켈스는 잘 씹은 담배를 한쪽 뺨에서 다른 쪽 뺨으로 옮겼다.",켈스는 특정 브랜드의 담배만 씹었다.,neutral
1,곡선 구조 위에 있는 익스트림 스포츠 스키 선수,스키 선수가 내리막길을 걷고 있다.,entailment
2,"반다나를 든 여자, 빨간 모자를 쓴 남자, 안경과 지팡이를 든 남자가 벤치에 앉아 ...",빨간 모자를 쓴 남자와 안경과 지팡이를 든 남자는 같은 사람이다.,contradiction
3,행사는 일본 사원에서 열린다.,그 행사는 야구장에서 열리고 있다.,contradiction
4,시간은 15분이다.,시간은 15분이었다.,entailment
...,...,...,...
941809,남자는 여자가 지켜보는 동안 한 잔에서 다른 잔으로 액체를 붓는 안전 유리가 달린 ...,한 잔에서 다른 잔으로 염산을 붓는 사람,neutral
941810,스케이트보드를 탄 소년이 비탈을 타고 있다.,한 소녀가 비탈에서 스케이트보드를 타고 있다.,contradiction
941811,어떤 이전 행정부에서도 이의를 제기하지 않았던 이 입장은 우리가 받는 많은 의회 요...,아무도 대리점 프로세스에 대한 검토를 요청하지 않는다.,contradiction
941812,피날레는 흉골과 갈비뼈의 존재에 대해 의심의 여지없이 청중에게 아무도 남기지 않았습니다.,대담한 사람은 아무도 흉골과 갈비뼈가 존재한다는 것을 의심하지 않았다.,entailment


In [24]:
val_data

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction
...,...,...,...
2485,피스티는 피즐처럼 중간 영어의 피스틴으로 시작되어 방귀를 뀌기 위해 주먹을 쥐었다.,Fiesty는 100년 동안 존재해 왔다.,neutral
2486,피스티는 피즐처럼 중간 영어의 피스틴으로 시작되어 방귀를 뀌기 위해 주먹을 쥐었다.,Fiesty는 주먹질과는 무관하다.,contradiction
2487,"진술이 더 나은 반면, 대답은 완성의 정신적 그림을 준다.",진술은 더 자세한 내용을 알려준다.,neutral
2488,"진술이 더 나은 반면, 대답은 완성의 정신적 그림을 준다.",진술이 더 나은 것은 아니다.,contradiction


In [25]:
test_data

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment
...,...,...,...
5005,데이비슨은 스콘의 발음을 '뼈'와 운을 맞추기 위해 채택해서는 안 된다. 어쨌든 그...,Davidson은 뼈와 스콘이 같은 소리를 내는 방식으로 말하면 안 된다.,entailment
5006,데이비슨은 스콘의 발음을 '뼈'와 운을 맞추기 위해 채택해서는 안 된다. 어쨌든 그...,Davidson이 스콘과 뼈라는 단어에 운율을 맞추는 편이 나을 것이다.,contradiction
5007,"25달러에 20만 단어의 평균 소설은 달러당 8,000단어로 적용된다.",25달러짜리 20만 단어 소설은 공정한 가격이다.,neutral
5008,"25달러에 20만 단어의 평균 소설은 달러당 8,000단어로 적용된다.",25달러짜리 20만 단어 소설은 달러당 4천 단어다.,contradiction


In [26]:
print('훈련용 샘플 개수 :',len(train_data))
print('검증용 샘플 개수 :',len(val_data))
print('테스트용 샘플 개수 :',len(test_data))

훈련용 샘플 개수 : 941814
검증용 샘플 개수 : 2490
테스트용 샘플 개수 : 5010


In [27]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [28]:
max_seq_len = 128

In [29]:
sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]

print(sent1)
print(sent2)

그도 최고야, 커비 켈스는 잘 씹은 담배를 한쪽 뺨에서 다른 쪽 뺨으로 옮겼다.
켈스는 특정 브랜드의 담배만 씹었다.


In [30]:
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'pad_to_max_length': True} not recognized.
Keyword arguments {'pad_to_max_length': True} not recognized.


In [31]:
print(encoding_result['input_ids'])

[2, 636, 2119, 3841, 2275, 16, 1710, 2151, 1721, 2255, 2259, 1521, 1373, 2073, 5840, 2138, 6986, 1208, 27135, 3656, 1613, 1208, 6233, 10047, 2062, 18, 3, 1721, 2255, 2259, 4814, 4183, 2079, 5840, 2154, 1373, 2359, 2062, 18, 3]


In [32]:
print(encoding_result['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
print(encoding_result['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [34]:
from transformers import logging
logging.set_verbosity_error()

def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids = [], [], []

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
        encoding_result = tokenizer.encode_plus(
            sent1,
            sent2,
            max_length=max_seq_len,
            padding='max_length',
            truncation=True,          # truncation 추가
            return_tensors='np'       # numpy array로 반환
        )

        input_ids.append(encoding_result['input_ids'][0])  # 배치 차원 제거
        attention_masks.append(encoding_result['attention_mask'][0])
        token_type_ids.append(encoding_result['token_type_ids'][0])

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    return (input_ids, attention_masks, token_type_ids)

In [35]:
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 941814/941814 [05:02<00:00, 3110.74it/s]


In [36]:
# 최대 길이: 128
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2   636  2119  3841  2275    16  1710  2151  1721  2255  2259  1521
  1373  2073  5840  2138  6986  1208 27135  3656  1613  1208  6233 10047
  2062    18     3  1721  2255  2259  4814  4183  2079  5840  2154  1373
  2359  2062    18     3     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [37]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 2490/2490 [00:00<00:00, 2896.28it/s]


In [38]:
# 최대 길이: 128
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  3673   636  2116  1041  2371  2062    16     6  4122    16  1535
  1458 10283    18     6     3   636  2259  3741  4942  2116   636  2138
  4105  2223  2155  6000  4122  2170  2318  4117  2138   572  2359  2062
    18     3     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [39]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 5010/5010 [00:01<00:00, 2981.78it/s]


In [40]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

In [41]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_val = idx_encode.transform(val_label) # 고유한 정수로 변환
y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{np.str_('contradiction'): np.int64(0), np.str_('entailment'): np.int64(1), np.str_('neutral'): np.int64(2)}
{np.int64(0): np.str_('contradiction'), np.int64(1): np.str_('entailment'), np.int64(2): np.str_('neutral')}


In [42]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [43]:

model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

I0000 00:00:1753949933.904167     293 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79194 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:84:00.0, compute capability: 8.0


In [44]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=5, batch_size=32, validation_data = (X_val, y_val),
    callbacks = [early_stopping]
)

Epoch 1/5


I0000 00:00:1753949950.075993    1255 service.cc:152] XLA service 0x70bb1400c4e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753949950.076036    1255 service.cc:160]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2025-07-31 08:19:10.371188: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-31 08:19:10.407760: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator tf_bert_for_sequence_classification_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
I0000 00:00:1753949951.882906    1255 cuda_dnn.cc:529] Loaded cuDNN version 90800
2025-07-31 08:19:14.772827: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 152 bytes spill stores, 152 bytes spill lo

    4/29432 ━━━━━━━━━━━━━━━━━━━━ 9:04 18ms/step - accuracy: 0.4258 - loss: 1.2448    

I0000 00:00:1753949958.350961    1255 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


29429/29432 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4640 - loss: 1.0341

2025-07-31 08:26:28.293425: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator tf_bert_for_sequence_classification_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2025-07-31 08:26:32.069569: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_13150', 8 bytes spill stores, 8 bytes spill loads

2025-07-31 08:26:32.678064: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_13150', 320 bytes spill stores, 320 bytes spill loads

2025-07-31 08:26:32.727247: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 656 bytes spill stores, 496 bytes spill loads

2025-07-31 08:26:32.741364: I external/local_xla/xla/stream_executor/cuda/subp

29432/29432 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4640 - loss: 1.0341

2025-07-31 08:26:37.836604: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator tf_bert_for_sequence_classification_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2025-07-31 08:26:42.009124: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator tf_bert_for_sequence_classification_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2025-07-31 08:26:44.527550: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_24', 64 bytes spill stores, 64 bytes spill loads

2025-07-31 08:26:44.888999: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6732', 8 bytes spill stores, 8 bytes spill loads

2025-07-31 08:26:45.330143: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : R

29432/29432 ━━━━━━━━━━━━━━━━━━━━ 466s 15ms/step - accuracy: 0.4919 - loss: 1.0077 - val_accuracy: 0.4695 - val_loss: 1.0334
Epoch 2/5
29432/29432 ━━━━━━━━━━━━━━━━━━━━ 433s 15ms/step - accuracy: 0.5270 - loss: 0.9725 - val_accuracy: 0.4851 - val_loss: 1.0174
Epoch 3/5
29432/29432 ━━━━━━━━━━━━━━━━━━━━ 432s 15ms/step - accuracy: 0.5360 - loss: 0.9622 - val_accuracy: 0.4968 - val_loss: 1.0121
Epoch 4/5
29432/29432 ━━━━━━━━━━━━━━━━━━━━ 396s 13ms/step - accuracy: 0.5409 - loss: 0.9564 - val_accuracy: 0.4948 - val_loss: 1.0083
Epoch 5/5
29432/29432 ━━━━━━━━━━━━━━━━━━━━ 407s 14ms/step - accuracy: 0.5438 - loss: 0.9526 - val_accuracy: 0.5048 - val_loss: 1.0062


In [47]:
# .keras 확장자 사용 (권장)
model.save('20250718_강의복습_KoBERT_KorNLI.keras')

In [48]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))

2025-07-31 08:56:10.551679: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator tf_bert_for_sequence_classification_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2025-07-31 08:56:13.244281: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6732', 44 bytes spill stores, 44 bytes spill loads

2025-07-31 08:56:13.261261: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 64 bytes spill stores, 64 bytes spill loads

2025-07-31 08:56:13.339681: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6732', 140 bytes spill stores, 140 bytes spill loads

2025-07-31 08:56:13.339716: I external/local_xla/xla/stream_executor/cuda/subproce

4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.5085 - loss: 0.9935

2025-07-31 08:56:17.766505: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator tf_bert_for_sequence_classification_1/tf_bert_model/bert/embeddings/assert_less/Assert/Assert
2025-07-31 08:56:20.395667: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 64 bytes spill stores, 64 bytes spill loads

2025-07-31 08:56:20.461405: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6732', 140 bytes spill stores, 140 bytes spill loads

2025-07-31 08:56:20.480368: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6732', 320 bytes spill stores, 320 bytes spill loads

2025-07-31 08:56:20.489911: I external/local_xla/xla/stream_executor/cuda/subpro

5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.4984 - loss: 1.0069

 테스트 정확도: 0.4984
